### 提示樣板

* 提示樣本其實就是把使用者與模型之間一問一答的過程，寫成一個像是**聊天紀錄的格式**，用來「欺騙」語言模型生成下個聊天片段。
* 這個樣板會根據每個語言模型訓練的方式而有所不同，一般而言遵守模型開發者的建議會比較好。

#### Llama2 提示樣板範例
* [Llama2 的官方樣板](https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGML/discussions/3#64b71f7588b86014d7e2dd71)：
    ```
    [INST] <使用者訊息> [/INST] <模型的回覆從這開始>
    ```

In [1]:
from transformers import TextStreamer
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM as ModelCls
from transformers import AutoTokenizer as TkCls

model_path = "TheBloke/Llama-2-7b-chat-fp16"
model: ModelCls = ModelCls.from_pretrained(
    model_path,
    device_map="auto",
    low_cpu_mem_usage=True,
    quantization_config=BitsAndBytesConfig(load_in_8bit=True),
)
tk: TkCls = TkCls.from_pretrained(model_path)

prompt = "[INST] 使用繁體中文回答，請問什麼是大型語言模型？ [/INST] "

inputs = tk(prompt, return_tensors="pt").to("cuda")
output = model.generate(
    **inputs,
    max_new_tokens=128,
    streamer=TextStreamer(tk),
)
# Output: <s> [INST] 使用繁體中文回答，請問什麼是大型語言模型？ [/INST] 大型語言模型（Large Language Model，LLM）是一種 ...</s>

/home/ai-x/miniconda3/envs/ann_py311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.62s/it]
/home/ai-x/miniconda3/envs/ann_py311/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/ai-x/miniconda3/envs/ann_py311/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. 

<s> [INST] 使用繁體中文回答，請問什麼是大型語言模型？ [/INST] 

/home/ai-x/miniconda3/envs/ann_py311/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/ai-x/miniconda3/envs/ann_py311/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


 Great! Here's my answer in Traditional Chinese:

大型語言模型 (大型语言模型) refers to a type of artificial intelligence (AI) model that is trained on a large corpus of text data to generate language outputs that are coherent and natural-sounding. These models are designed to mimic the language abilities of humans, such as understanding context, using grammar and syntax, and producing text that is grammatically correct and contextually appropriate.
大型語言模型 can be used for a variety of applications, such as:
1. Language Translation


#### 直接調用行開發者設定的模板： (樣板隨模型而變化)
* 使用 Transformer 套件內建的**聊天樣板 (Chat Template)** 功能，只要呼叫分詞器的 `apply_chat_template` 方法即可。
* Google 的 Gemma 提示樣板範例

 1. 呼叫分詞器的 `apply_chat_template` 方法：

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_path = "google/gemma-2b-it"
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto")
tk = AutoTokenizer.from_pretrained(model_path)

query = " 使用繁體中文回答，請問什麼是大型語言模型？ "
conversation = [{"role": "user", "content": query}]
prompt = tk.apply_chat_template(conversation, tokenize=False)
print(prompt)

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.35s/it]


<bos><start_of_turn>user
使用繁體中文回答，請問什麼是大型語言模型？<end_of_turn>



2. 附加上可以把模型開始回答前的格式，讓模型比較不容易生成失敗。(透過 `add_generation_prompt` 參數)

In [3]:

# model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
# tk = AutoTokenizer.from_pretrained(model_path)
# query = " 使用繁體中文回答，請問什麼是大型語言模型？ "
# conversation = [{"role": "user", "content": query}]
prompt = tk.apply_chat_template(
    conversation, 
    tokenize=False,
    add_generation_prompt=True,
)
print(prompt)

<bos><start_of_turn>user
使用繁體中文回答，請問什麼是大型語言模型？<end_of_turn>
<start_of_turn>model



3. 格式確認完畢後，來取得模型的輸入：

In [4]:
input_ids = tk.apply_chat_template(
    conversation, 
    # tokenize=False,
    return_tensors="pt",
    add_generation_prompt=True,
).to("cuda")
print(input_ids)

# Output: tensor([[2, 106, ..., 2516, 108]], device='cuda:0')

tensor([[     2,    106,   1645,    108,   7060, 238249, 236969,  50039,  33226,
         235365, 160287,  24437, 235427,  66240, 144955,  43936, 235544,    107,
            108,    106,   2516,    108]], device='cuda:0')


4. 輸入模型

In [6]:
# output = model.generate(**inputs) # 原本
output = model.generate(
    input_ids,
    max_new_tokens=2048,
    streamer=TextStreamer(tk),
)  # 現在

<bos><start_of_turn>user
使用繁體中文回答，請問什麼是大型語言模型？<end_of_turn>
<start_of_turn>model
大型語言模型（LLM）是一種 AI 模型，它具有超過 1 公億個字的語言資料，並能夠根據這些資料進行語言處理。這使得它能夠完成各種任務，包括：

* 翻譯文字
* 產生文字
* 識別語言
* 進行語言推理
* 識別情緒

LLM 的出現改變了 AI 的發展方向，因為它們可以被用來解決各種問題，包括：

* 減少語言溝通中的歧視
* 提高語言模型的解釋性
* 建立更可靠的語言模型

目前，LLM 是各種語言模型的基礎，包括：

* **BERT**
* **GPT**
* **XLNet**
* **T5**

LLM 的發展是一個不斷的過程，隨著新的模型被開發出來，它們變得越來越精確和高效。<eos>
